# Performance

Obviously, using unit-aware variables will slow down any computation compared to raw python values (int, flot, numpy.ndarray).

In [6]:
import matplotlib.pyplot as plt
import numpy as np

import physipy
from physipy import s, m, setup_matplotlib

from physipy import Dimension, units, quantify, Quantity



ms = units["ms"]
mm = units['mm']
km = units["km"]
cm = units["cm"]
mus = units["mus"]
ns = units["ns"]
a = 123456
b = 654321

aq = a*m
bq = b*m


Basic comparison on addition

In [10]:
%timeit  (a +  b)
%timeit (aq + bq)

63.7 ns ± 2.37 ns per loop (mean ± std. dev. of 7 runs, 10000000 loops each)
12.4 µs ± 446 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [14]:
print(12.4*mus/(63.7*ns))

194.66248037676607


Basic comparison on pow

In [12]:
%timeit  (a**2)
%timeit (aq**2)

289 ns ± 22.7 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)
22.8 µs ± 989 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [15]:
print(22.8*mus/(289*ns))

78.89273356401384


## benchmark timing

Here is a comparison of most operations : 

In [32]:
import timeit

operations = {
    "add":"__add__", 
    #"sub":"__sub__",
    #"mul":"__mul__",
}

a = 123456
b = 654321

aq = a*m
bq = b*m

for operation, operation_method in operations.items():
    time = timeit.timeit('a.'+operation_method+"(b)", number=10000, globals=globals())
    time_q = timeit.timeit('aq.'+operation_method+"(bq)", number=10000, globals=globals())
    print(f"{m: >5} : {time_q/time: <5.1f}")

  add : 1802.0


# Dimension performance

Importance :
 - Addition of values key-wise
 - Substration of values key-wise
 - Multiply values
 

In [2]:
import numpy as np
import collections
"""Goal : return True if 2 vectors of numbers are equal
Inputs :
 - vectors are assured to be the same size
 - vector values can be int, float, np.numbers, fractions
 - the order of the numbers matters (like with dict comparison or ordered dict)
"""
 
as_dictl = {"A":0, "B":0, "C":0}
as_dictr = {"A":0, "B":0, "C":0}
as_listl = [0, 0, 0]
as_listr = [0, 0, 0]
as_arryl = np.array([0, 0, 0])
as_arryr = np.array([0, 0, 0])
as_odictl = collections.OrderedDict( {"A":0, "B":0, "C":0})
as_odictr = collections.OrderedDict( {"A":0, "B":0, "C":0})

%timeit as_listl == as_listr
%timeit as_dictl == as_dictr
%timeit as_odictl == as_odictr
%timeit as_arryl.tolist() == as_arryr.tolist()
%timeit list(as_odictl.values()) == list(as_odictr.values())
%timeit np.array_equal(as_arryl, as_arryr)
%timeit np.all(as_arryl == as_arryr)

60.2 ns ± 6.61 ns per loop (mean ± std. dev. of 7 runs, 10000000 loops each)
89.1 ns ± 16 ns per loop (mean ± std. dev. of 7 runs, 10000000 loops each)
134 ns ± 13.7 ns per loop (mean ± std. dev. of 7 runs, 10000000 loops each)
367 ns ± 18.7 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)
894 ns ± 53.5 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)
4.7 µs ± 257 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)
6.66 µs ± 1.04 µs per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [12]:
import numpy as np
import collections
from operator import add


as_dictl = {"A":0, "B":0, "C":0}
as_dictr = {"A":0, "B":0, "C":0}
as_listl = [0, 0, 0]
as_listr = [0, 0, 0]
as_arryl = np.array([0, 0, 0])
as_arryr = np.array([0, 0, 0])
as_odictl = collections.OrderedDict( {"A":0, "B":0, "C":0})
as_odictr = collections.OrderedDict( {"A":0, "B":0, "C":0})

%timeit [l+r for l,r in zip(as_listl, as_listr)]
%timeit {k:as_dictl[k]+as_dictr[k] for k in (as_dictl.keys() & as_dictr.keys())}
#%timeit as_odictl == as_odictr
#%timeit as_arryl.tolist() == as_arryr.tolist()
#%timeit list(as_odictl.values()) == list(as_odictr.values())
#%timeit np.array_equal(as_arryl, as_arryr)
%timeit as_arryl + as_arryr
%timeit list(map(add, as_listl, as_listr))

616 ns ± 27.4 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)
1.35 µs ± 264 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)
600 ns ± 45.5 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)
624 ns ± 61 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [11]:
import numpy as np
import collections
from operator import mul

as_dictl = {"A":0, "B":0, "C":0}
as_dictr = {"A":0, "B":0, "C":0}
as_listl = [0, 0, 0]
as_listr = [0, 0, 0]
as_arryl = np.array([0, 0, 0])
as_arryr = np.array([0, 0, 0])
as_odictl = collections.OrderedDict( {"A":0, "B":0, "C":0})
as_odictr = collections.OrderedDict( {"A":0, "B":0, "C":0})

%timeit [l*r for l,r in zip(as_listl, as_listr)]
%timeit {k:as_dictl[k]*as_dictr[k] for k in (as_dictl.keys() & as_dictr.keys())}
#%timeit as_odictl == as_odictr
#%timeit as_arryl.tolist() == as_arryr.tolist()
#%timeit list(as_odictl.values()) == list(as_odictr.values())
#%timeit np.array_equal(as_arryl, as_arryr)
%timeit as_arryl * as_arryr
%timeit list(map(mul, as_listl, as_listr))

685 ns ± 145 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)
1.15 µs ± 87.4 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)
544 ns ± 50.9 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)
574 ns ± 36.5 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)
